# OSRS hiscores example plots

This notebook reads in example hiscores data and produces plots similar to those produced by the OSRS_hiscores.py script, the purpose being to give a quick example of the type of plots output by said script.

The plots produced are saved and already available in the `examples/` directory.

---

### Imports, styles and jupyter notebook requirements

In [1]:
import pandas as pd
from plotly.subplots import make_subplots
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches

%matplotlib inline
init_notebook_mode(connected=True)
plt.style.use('ggplot')

---

### Read in example data

In [2]:
hiscores_all_time = pd.read_csv('example-player-hiscores.csv', parse_dates=[0],
                                names=['Date', 'Skill', 'Rank', 'Level', 'XP'])

In [3]:
hiscores_all_time

,Date,Skill,Rank,Level,XP
0,2022-04-14 16:30:00,Overall,53398,2183,262059444
1,2022-04-14 16:30:00,Attack,224348,99,13071871
2,2022-04-14 16:30:00,Defence,280705,94,8146484
3,2022-04-14 16:30:00,Strength,158099,99,15203744
4,2022-04-14 16:30:00,Hitpoints,105583,99,26615385
...,...,...,...,...,...
370,2022-04-18 16:31:00,Vet'ion,16163,100,100
371,2022-04-18 16:31:00,Vorkath,63130,800,800
372,2022-04-18 16:31:00,Wintertodt,261896,253,253
373,2022-04-18 16:31:00,Zalcano,43438,160,160


---

### Split dataframe into "skills" and "kill count" and create "total kill count" dataframe

In [4]:
# Create dataframe containing only "skills"
skill_list = ['Overall', 'Attack', 'Defence', 'Strength', 'Hitpoints',
              'Ranged', 'Prayer', 'Magic', 'Cooking', 'Woodcutting',
              'Fletching', 'Fishing', 'Firemaking', 'Crafting', 'Smithing',
              'Mining', 'Herblore', 'Agility', 'Thieving', 'Slayer',
              'Farming', 'Runecraft', 'Hunter', 'Construction']
single_skills = []
for s in skill_list:
    single_skill = hiscores_all_time[(hiscores_all_time['Skill'] == s)]
    single_skills.append(single_skill)
skills = pd.concat(single_skills)

In [5]:
# Create dataframe containing only "kill count" by removing skill rows
killcount = pd.merge(hiscores_all_time, skills, indicator=True,
                     how='outer').query('_merge=="left_only"')
killcount.drop(['_merge', 'XP'], axis=1, inplace=True)
killcount.columns = ['Date', 'Boss', 'Rank', 'Kill count']

In [6]:
# Create dataframe containing date and total kill count
unique_dates = list(killcount['Date'].unique())
total_killcount = [sum(killcount[killcount['Date'] == date]['Kill count'])
                   for date in unique_dates]
total_killcount = pd.DataFrame({'Date': unique_dates,
                                'Kill count': total_killcount})

---

### Create interactive plots of overall level/XP and total boss kill count

In [7]:
overall_plot = make_subplots(rows=2, cols=2,
                             vertical_spacing=0.2,
                             specs=[[{}, {}],
                                    [{"colspan": 2}, None]])
overall_plot.append_trace(go.Scatter(x=skills[(skills['Skill'] ==
                                               'Overall')]['Date'],
                                     y=skills[(skills['Skill'] ==
                                               'Overall')]['Level'],
                                     name='Level', mode='lines'),
                          row=1, col=1)
overall_plot.append_trace(go.Scatter(x=skills[(skills['Skill'] ==
                                               'Overall')]['Date'],
                                     y=skills[(skills['Skill'] ==
                                               'Overall')]['XP'],
                                     name='XP', mode='lines'),
                          row=1, col=2)
overall_plot.append_trace(go.Scatter(x=total_killcount['Date'],
                                     y=total_killcount['Kill count'],
                                     name='Kill count', mode='lines'),
                          row=2, col=1)
overall_plot.update_xaxes(title_text='Date', row=1, col=1)
overall_plot.update_xaxes(title_text='Date', row=1, col=2)
overall_plot.update_xaxes(title_text='Date', row=2, col=1)
overall_plot.update_yaxes(title_text='Total level', row=1, col=1)
overall_plot.update_yaxes(title_text='Total XP', row=1, col=2)
overall_plot.update_yaxes(title_text='Total boss kill count', row=2, col=1)
overall_plot.update_layout(title_text='Overall stats')
iplot(overall_plot) # File saved manually as plotly_overall_stats.png

---

### Plot each skill on figure in facetgrid

In [8]:
def RotateTickLabels(fig):
    """
    Rotates all x-axis tick labels in seaborn facetgrid

    Parameters
    ----------
    fig : seaborn facetgrid
    """
    for axes in fig.axes.flat:
        _ = axes.set_xticklabels(axes.get_xticklabels(), rotation=90)

In [ ]:
# Plot skills on facetgrid (one skill per plot)
skills_plot = sns.FacetGrid(data=skills, col='Skill', col_wrap=4,
                            sharey=False, height=3.5, aspect=1.5)
skills_plot.map(sns.lineplot, 'Date', 'XP')
# Add plot of level to XP plot using a 2nd y-axis
for ax, (_, subdata) in zip(skills_plot.axes, skills.groupby('Skill', sort=False)):
    ax2 = ax.twinx()
    subdata.plot(x='Date', y='Level', ax=ax2, legend=False, color='b')
    ax.set_ylabel('XP')
    ax2.set_ylabel('Level')

    # Create legend (colours obtained via plt.gca().lines[-1].get_color())
    name_to_color = {
        'XP': (0.8862745098039215, 0.2901960784313726, 0.2),
        'Level': 'b'
    }
    patches = [matplotlib.patches.Patch(color=v, label=k) for k, v in name_to_color.items()]
    plt.legend(handles=patches, loc='upper left')
RotateTickLabels(skills_plot)
plt.tight_layout()
plt.savefig('seaborn_facetgrid_skills.png')

---

### Plot each boss kill count on figure in facetgrid

In [ ]:
killcount_plot = sns.FacetGrid(data=killcount, col='Boss', col_wrap=6,
                               sharey=False)
killcount_plot.map(sns.lineplot, 'Date', 'Kill count')
RotateTickLabels(killcount_plot)
plt.tight_layout()
plt.savefig('seaborn_facetgrid_killcount.png')